# MA_DL : Codierung 64bit

In [28]:
!pip install facenet-pytorch # due to issues with the google cloud service

In [29]:
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models
from PIL import Image

from facenet_pytorch import MTCNN, InceptionResnetV1

dtype = torch.cuda.FloatTensor

## Model

In [30]:
class MultiLabel(nn.Sequential):
    def __init__(self, input_dim=512, output_dim=64):
        super(MultiLabel, self).__init__()
        self.l1 = nn.Linear(input_dim, input_dim)
        self.l2 = nn.Linear(input_dim, input_dim)
        self.l5 = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l5(x)
        return torch.sigmoid(x)

In [33]:
model = MultiLabel()
#model.load_state_dict(torch.load('dl64b_v3_1000.pt'))
model.cuda()

MultiLabel(
  (l1): Linear(in_features=512, out_features=512, bias=True)
  (l2): Linear(in_features=512, out_features=512, bias=True)
  (l5): Linear(in_features=512, out_features=64, bias=True)
)

## Data

In [34]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()
to_image = transforms.ToPILImage()
transform = transforms.Compose([
    scaler, 
    to_tensor,
#    normalize
])

In [35]:
train_ds = datasets.ImageFolder('../../data/small_data/train/', transform=transform)
train_ds.idx_to_class = {i:c for c, i in train_ds.class_to_idx.items()}
train_dl = DataLoader(train_ds, batch_size=1, num_workers=4)

valid_ds = datasets.ImageFolder('../../data/small_data/valid/', transform=transform)
valid_ds.idx_to_class = {i:c for c, i in valid_ds.class_to_idx.items()}
valid_dl = DataLoader(valid_ds, batch_size=1, num_workers=4)

## Gesichtserkennung

In [36]:
mtcnn = MTCNN(
    image_size=160,
    thresholds=[0.6, 0.7, 0.7],
#        factor=0.709,
#        prewhiten=True,
    keep_all=True,
    device=torch.device('cuda')
    )

## Embeddings 

In [37]:
# Alternativ resnet
face_resnet = InceptionResnetV1(pretrained='casia-webface').eval()

## Training

In [38]:
def calculate_mistakes(pred, target):
    #pred = pred[0]
#    print('calculating mistakes : ')
#    print('   pred: ' + str(pred.shape))
#    print('   target: ' + str(target.shape))
    
    mistakes = 0
    
    if len(pred) != len(target):
        raise Exception('sizes of both tensors must match')
        
    for x,y in zip(pred, target):
        if round(x.item()) != y.item():
            mistakes = mistakes + 1
        
    return mistakes

In [39]:
loss_func = nn.BCELoss() #MultiLabelSoftMarginLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-5, momentum=0.9) #weight_decay=1e-4 

In [40]:
import matplotlib.pyplot as plt

In [41]:
epochs = 1000
for e in range(epochs):
    print("======================")
    print("Epoch : " + str(e))
    epoch_loss = 0
    epoch_mistakes = 0
    train_size = len(train_dl)
    
    # activate train mode
    model.train()

    for index, (data, target) in enumerate(train_dl):
        
        faces, prob = mtcnn(to_image(data[0]), return_prob=True)
                
        target_t = torch.cuda.FloatTensor([int(x) for x in train_ds.idx_to_class[target[0].item()]])

        emb = face_resnet(faces)    
        data_v   = Variable(emb[0], requires_grad=False).type(dtype)
        target_v = Variable(target_t, requires_grad=False).type(dtype)
        
        # forward
#        print('datav: ' + str(data_v.shape))
        pred = model.forward(data_v)
#        print('pred: ' + str(pred.shape))
        # zero grads
        optimizer.zero_grad()
        # calculate loss
        loss = loss_func(pred, target_v.float())
        epoch_loss = epoch_loss + loss
        # back prop
        loss.backward()
        optimizer.step()
        
        epoch_mistakes = epoch_mistakes + calculate_mistakes(pred, target_t)
        
    print("Train loss : " + str(epoch_loss.item()))
    print("Average errors : " + str(epoch_mistakes/train_size))
    
    # activate eval mode
    model.eval()
    
    valid_loss = 0
    valid_mistakes = 0
    valid_size = len(valid_dl)
    for index, (data, target) in enumerate(valid_dl):
        faces, prob = mtcnn(to_image(data[0]), return_prob=True)
        target_t = torch.FloatTensor([int(x) for x in valid_ds.idx_to_class[target[0].item()]])
        
        emb = face_resnet(faces) 
        data_v   = Variable(emb[0], requires_grad=False).type(dtype)
        target_v = Variable(target_t, requires_grad=False).type(dtype)

        pred = model.forward(data_v)
        loss = loss_func(pred, target_v.float())
        valid_loss = valid_loss + loss
        valid_mistakes = valid_mistakes + calculate_mistakes(pred, target_t)
        
        
    print("Valid loss : " + str(valid_loss.item()))
    print("Average valid errors : " + str(valid_mistakes/valid_size))

Epoch : 0
Train loss : 130.38844299316406
Average errors : 10.532786885245901
Valid loss : 14.902487754821777
Average valid errors : 9.952380952380953
Epoch : 1
Train loss : 129.74183654785156
Average errors : 10.453551912568306
Valid loss : 14.627086639404297
Average valid errors : 9.547619047619047
Epoch : 2
Train loss : 128.0278778076172
Average errors : 10.147540983606557
Valid loss : 14.431962966918945
Average valid errors : 9.166666666666666
Epoch : 3
Train loss : 126.50144958496094
Average errors : 9.980874316939891
Valid loss : 14.252883911132812
Average valid errors : 8.738095238095237
Epoch : 4
Train loss : 125.01726531982422
Average errors : 9.808743169398907
Valid loss : 14.10942554473877
Average valid errors : 8.571428571428571
Epoch : 5
Train loss : 123.5956039428711
Average errors : 9.699453551912569
Valid loss : 13.945332527160645
Average valid errors : 8.404761904761905
Epoch : 6
Train loss : 122.1824951171875
Average errors : 9.557377049180328
Valid loss : 13.79398345

Valid loss : 7.127933979034424
Average valid errors : 3.5952380952380953
Epoch : 48
Train loss : 71.06137084960938
Average errors : 4.617486338797814
Valid loss : 6.983773708343506
Average valid errors : 3.5238095238095237
Epoch : 49
Train loss : 70.0057601928711
Average errors : 4.543715846994536
Valid loss : 6.844700813293457
Average valid errors : 3.4523809523809526
Epoch : 50
Train loss : 68.96778106689453
Average errors : 4.461748633879782
Valid loss : 6.700443267822266
Average valid errors : 3.3333333333333335
Epoch : 51
Train loss : 67.92652893066406
Average errors : 4.385245901639344
Valid loss : 6.573859691619873
Average valid errors : 3.3333333333333335
Epoch : 52
Train loss : 66.91092681884766
Average errors : 4.300546448087432
Valid loss : 6.4355340003967285
Average valid errors : 3.1666666666666665
Epoch : 53
Train loss : 65.87431335449219
Average errors : 4.1775956284153
Valid loss : 6.303387641906738
Average valid errors : 3.0238095238095237
Epoch : 54
Train loss : 64.86

Train loss : 30.946613311767578
Average errors : 1.0655737704918034
Valid loss : 2.4741053581237793
Average valid errors : 0.5476190476190477
Epoch : 96
Train loss : 30.33373260498047
Average errors : 1.0163934426229508
Valid loss : 2.417978048324585
Average valid errors : 0.5476190476190477
Epoch : 97
Train loss : 29.72549819946289
Average errors : 0.9808743169398907
Valid loss : 2.361597776412964
Average valid errors : 0.5238095238095238
Epoch : 98
Train loss : 29.14360809326172
Average errors : 0.9480874316939891
Valid loss : 2.3056840896606445
Average valid errors : 0.5238095238095238
Epoch : 99
Train loss : 28.549558639526367
Average errors : 0.907103825136612
Valid loss : 2.253551959991455
Average valid errors : 0.5238095238095238
Epoch : 100
Train loss : 27.968460083007812
Average errors : 0.8797814207650273
Valid loss : 2.2014782428741455
Average valid errors : 0.42857142857142855
Epoch : 101
Train loss : 27.41164779663086
Average errors : 0.8524590163934426
Valid loss : 2.1532

Valid loss : 0.8591344356536865
Average valid errors : 0.023809523809523808
Epoch : 142
Train loss : 11.962836265563965
Average errors : 0.06557377049180328
Valid loss : 0.8404494524002075
Average valid errors : 0.023809523809523808
Epoch : 143
Train loss : 11.737247467041016
Average errors : 0.060109289617486336
Valid loss : 0.8242777585983276
Average valid errors : 0.023809523809523808
Epoch : 144
Train loss : 11.519392013549805
Average errors : 0.0546448087431694
Valid loss : 0.8075026869773865
Average valid errors : 0.023809523809523808
Epoch : 145
Train loss : 11.305469512939453
Average errors : 0.05191256830601093
Valid loss : 0.791562557220459
Average valid errors : 0.023809523809523808
Epoch : 146
Train loss : 11.09619426727295
Average errors : 0.04918032786885246
Valid loss : 0.7762175798416138
Average valid errors : 0.023809523809523808
Epoch : 147
Train loss : 10.892608642578125
Average errors : 0.03825136612021858
Valid loss : 0.7605792880058289
Average valid errors : 0.0
E

KeyboardInterrupt: 

In [42]:
torch.save(model.state_dict(), 'dl64b_v3_1170.pt')

## Testing 

In [62]:
def tensor_to_binary(tensor):
    if tensor.dim() != 2:
        raise ValueException('tensor must have two dimensions')

    for i, x in enumerate(tensor):
        for j, y in enumerate(x) :
            if y.item() < 0.5:
                tensor[i][j] = 0
            elif y.item() >= 0.5:
                tensor[i][j] = 1

    return tensor

In [65]:
for i, t in train_ds:
    print(i.shape)
    faces, prob = mtcnn(to_image(i), return_prob=True)
    target_t = torch.FloatTensor([int(x) for x in valid_ds.idx_to_class[target[0].item()]])
        
    emb = face_resnet(faces) 
    data_v   = Variable(emb[0], requires_grad=False).type(dtype)
    target_v = Variable(target_t, requires_grad=False).type(dtype)

    pred = model.forward(data_v)
    print(pred.shape)
    print(tensor_to_binary(pred.unsqueeze(0)))

torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1.

torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 

torch.Size([64])
tensor([[0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
         1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1.,
         1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         0., 1., 0., 0., 1., 0., 1., 1., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 1., 1., 1., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 0., 1., 1., 1., 1., 1., 1., 

torch.Size([64])
tensor([[0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1.,
         0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
         0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0.,
         0., 1., 0., 1., 1., 0., 0., 1., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
         0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 1., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
         0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 1., 

torch.Size([64])
tensor([[0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
         1., 0., 0., 1., 0., 1., 0., 1., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
         1., 1., 0., 1., 0., 1., 0., 0., 1., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 1., 1., 1., 

...

torch.Size([64])
tensor([[1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
         0., 1., 0., 0., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
         0., 1., 0., 0., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 0., 

torch.Size([64])
tensor([[1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
         0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
         0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1.,
         1., 0., 0., 1., 0., 0., 0., 0., 

torch.Size([64])
tensor([[1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
         0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
         0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
         1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
         1., 1., 1., 0., 0., 0., 0., 0., 

torch.Size([64])
tensor([[1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 1., 0., 0., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 1., 0., 0., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
         0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
         0., 0., 0., 1., 0., 1., 1., 1., 

torch.Size([64])
tensor([[1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 0., 0., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 0., 0., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 

In [47]:
!ls ../../data/small_data/train/1111100010001101110100100001010010101001001010001110110110000011

Tom_Hanks_0001.jpg  Tom_Hanks_0004.jpg	Tom_Hanks_0007.jpg  Tom_Hanks_0010.jpg
Tom_Hanks_0002.jpg  Tom_Hanks_0005.jpg	Tom_Hanks_0008.jpg
Tom_Hanks_0003.jpg  Tom_Hanks_0006.jpg	Tom_Hanks_0009.jpg


In [ ]:
path = '../../data/small_data/train/1111100010001101110100100001010010101001001010001110110110000011/'
for i in os.listdir(path):
        Image.open(path + i)
Image.open('../../data/small_data/train/1111100010001101110100100001010010101001001010001110110110000011/T')

In [ ]:
model.forward()